# 可視化とグラフ分析：確率計画法の結果理解

このノートブックでは、確率計画法の結果を効果的に可視化し、意思決定支援のためのインサイトを抽出します。

## 学習目標

1. 確率分布と不確実性の可視化
2. 最適化結果の多次元分析
3. リスク-リターン分析
4. 感度分析の可視化
5. インタラクティブな分析ツール
6. 意思決定支援ダッシュボード

In [ ]:
# 必要ライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# matplotlib設定
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 11
plt.rcParams['axes.grid'] = True

# seaborn設定
sns.set_palette("husl")

print("可視化ライブラリのインポート完了")
print(f"Matplotlib: {plt.matplotlib.__version__}")
print(f"Seaborn: {sns.__version__}")
print(f"Plotly: {go.__version__}")

## 1. 基本的な確率分布の可視化

In [ ]:
def create_comprehensive_distribution_plots():
    """
    包括的な確率分布可視化
    """
    # データ生成
    np.random.seed(42)
    
    # 複数の分布を比較
    x = np.linspace(0, 100, 1000)
    
    distributions = {
        'Normal(50,10)': stats.norm(50, 10),
        'Normal(50,5)': stats.norm(50, 5),
        'Uniform(30,70)': stats.uniform(30, 40),
        'Beta(scaled)': stats.beta(2, 3, loc=30, scale=40)
    }
    
    # 静的グラフ（matplotlib + seaborn）
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. PDF比較
    colors = ['blue', 'red', 'green', 'purple']
    for i, (name, dist) in enumerate(distributions.items()):
        pdf_values = dist.pdf(x)
        ax1.plot(x, pdf_values, color=colors[i], linewidth=2, label=name, alpha=0.8)
        ax1.fill_between(x, pdf_values, alpha=0.2, color=colors[i])
    
    ax1.set_xlabel('Wind Output [MW]')
    ax1.set_ylabel('Probability Density')
    ax1.set_title('Probability Density Functions Comparison')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. CDF比較
    for i, (name, dist) in enumerate(distributions.items()):
        cdf_values = dist.cdf(x)
        ax2.plot(x, cdf_values, color=colors[i], linewidth=2, label=name)
    
    ax2.set_xlabel('Wind Output [MW]')
    ax2.set_ylabel('Cumulative Probability')
    ax2.set_title('Cumulative Distribution Functions')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. サンプルヒストグラム
    n_samples = 10000
    sample_data = []
    
    for i, (name, dist) in enumerate(distributions.items()):
        samples = dist.rvs(n_samples)
        ax3.hist(samples, bins=50, alpha=0.6, label=name, color=colors[i], density=True)
        sample_data.append({'Distribution': name, 'Samples': samples})
    
    ax3.set_xlabel('Wind Output [MW]')
    ax3.set_ylabel('Density')
    ax3.set_title('Sample Distributions (10,000 samples each)')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Box Plot比較
    sample_df = pd.DataFrame()
    for item in sample_data:
        temp_df = pd.DataFrame({
            'Distribution': item['Distribution'],
            'Value': item['Samples']
        })
        sample_df = pd.concat([sample_df, temp_df], ignore_index=True)
    
    sns.boxplot(data=sample_df, x='Distribution', y='Value', ax=ax4)
    ax4.set_title('Distribution Comparison (Box Plots)')
    ax4.set_ylabel('Wind Output [MW]')
    ax4.tick_params(axis='x', rotation=45)
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 統計量比較表
    stats_comparison = []
    for name, dist in distributions.items():
        samples = dist.rvs(10000)
        stats_comparison.append({
            'Distribution': name,
            'Mean': np.mean(samples),
            'Std': np.std(samples),
            'Skewness': stats.skew(samples),
            'Kurtosis': stats.kurtosis(samples),
            'VaR_95': np.percentile(samples, 5),
            'VaR_99': np.percentile(samples, 1)
        })
    
    stats_df = pd.DataFrame(stats_comparison)
    print("\n=== 分布統計量比較 ===")
    display(stats_df.round(2))
    
    return sample_df

# 実行
distribution_data = create_comprehensive_distribution_plots()

## 2. インタラクティブな最適化結果可視化

In [ ]:
def create_interactive_optimization_dashboard():
    """
    Plotlyを使用したインタラクティブな最適化ダッシュボード
    """
    # 最適化結果データの生成（前のノートブックの結果を模擬）
    np.random.seed(42)
    
    # パラメータ範囲
    wind_means = np.linspace(30, 70, 20)
    emergency_costs = np.linspace(100, 300, 15)
    
    # 最適化結果のグリッド生成
    results_data = []
    
    for wind_mean in wind_means:
        for emergency_cost in emergency_costs:
            # 簡化された最適化計算
            demand = 80
            conv_cost = 60
            wind_std = 10
            
            # 解析的近似解
            optimal_x = demand - wind_mean + wind_std * np.sqrt(2 * np.log(emergency_cost / conv_cost))
            optimal_x = max(0, min(100, optimal_x))  # 制約内に収める
            
            # 期待コスト計算
            expected_cost = conv_cost * optimal_x + emergency_cost * 0.1  # 簡化
            
            # VaR/CVaR推定
            var_95 = expected_cost * 1.2
            cvar_95 = expected_cost * 1.35
            
            results_data.append({
                'Wind_Mean': wind_mean,
                'Emergency_Cost': emergency_cost,
                'Optimal_Generation': optimal_x,
                'Expected_Cost': expected_cost,
                'VaR_95': var_95,
                'CVaR_95': cvar_95,
                'Risk_Premium': cvar_95 - expected_cost
            })
    
    results_df = pd.DataFrame(results_data)
    
    # インタラクティブ3D散布図
    fig1 = go.Figure(data=go.Scatter3d(
        x=results_df['Wind_Mean'],
        y=results_df['Emergency_Cost'],
        z=results_df['Optimal_Generation'],
        mode='markers',
        marker=dict(
            size=5,
            color=results_df['Expected_Cost'],
            colorscale='Viridis',
            colorbar=dict(title="Expected Cost [$]")
        ),
        text=[f'Wind: {w:.1f}MW<br>Emergency: ${e:.0f}<br>Generation: {g:.1f}MW<br>Cost: ${c:.0f}'
              for w, e, g, c in zip(results_df['Wind_Mean'], results_df['Emergency_Cost'],
                                   results_df['Optimal_Generation'], results_df['Expected_Cost'])],
        hovertemplate='%{text}<extra></extra>'
    ))
    
    fig1.update_layout(
        title='3D Optimization Landscape',
        scene=dict(
            xaxis_title='Wind Mean [MW]',
            yaxis_title='Emergency Cost [$/MWh]',
            zaxis_title='Optimal Generation [MW]'
        ),
        width=800,
        height=600
    )
    
    fig1.show()
    
    # ヒートマップ
    pivot_optimal = results_df.pivot(index='Emergency_Cost', columns='Wind_Mean', values='Optimal_Generation')
    pivot_cost = results_df.pivot(index='Emergency_Cost', columns='Wind_Mean', values='Expected_Cost')
    
    fig2 = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Optimal Generation [MW]', 'Expected Cost [$]'),
        horizontal_spacing=0.1
    )
    
    # 最適発電量ヒートマップ
    fig2.add_trace(
        go.Heatmap(
            z=pivot_optimal.values,
            x=pivot_optimal.columns,
            y=pivot_optimal.index,
            colorscale='RdYlBu_r',
            colorbar=dict(title="Generation [MW]", x=0.45)
        ),
        row=1, col=1
    )
    
    # 期待コストヒートマップ
    fig2.add_trace(
        go.Heatmap(
            z=pivot_cost.values,
            x=pivot_cost.columns,
            y=pivot_cost.index,
            colorscale='Viridis',
            colorbar=dict(title="Cost [$]", x=1.02)
        ),
        row=1, col=2
    )
    
    fig2.update_xaxes(title_text="Wind Mean [MW]", row=1, col=1)
    fig2.update_xaxes(title_text="Wind Mean [MW]", row=1, col=2)
    fig2.update_yaxes(title_text="Emergency Cost [$/MWh]", row=1, col=1)
    fig2.update_yaxes(title_text="Emergency Cost [$/MWh]", row=1, col=2)
    
    fig2.update_layout(title='Parameter Sensitivity Heatmaps', width=1000, height=500)
    fig2.show()
    
    return results_df

# インタラクティブダッシュボード作成
optimization_results = create_interactive_optimization_dashboard()

## 3. リスク分析の高度な可視化

In [ ]:
def create_advanced_risk_analysis():
    """
    高度なリスク分析可視化
    """
    # リスク分析用データ生成
    np.random.seed(123)
    n_scenarios = 1000
    
    # 複数の戦略
    strategies = {
        'Conservative': {'gen': 45, 'color': 'blue'},
        'Moderate': {'gen': 35, 'color': 'green'},
        'Aggressive': {'gen': 25, 'color': 'red'}
    }
    
    # 各戦略のコスト分布生成
    wind_scenarios = np.random.normal(50, 10, n_scenarios)
    demand = 80
    conv_cost = 60
    emergency_cost = 200
    
    strategy_results = {}
    
    for strategy_name, strategy_params in strategies.items():
        gen = strategy_params['gen']
        
        # 各シナリオでのコスト計算
        shortage = np.maximum(0, demand - gen - wind_scenarios)
        total_costs = conv_cost * gen + emergency_cost * shortage
        
        # リスク指標計算
        mean_cost = np.mean(total_costs)
        std_cost = np.std(total_costs)
        var_95 = np.percentile(total_costs, 95)
        var_99 = np.percentile(total_costs, 99)
        cvar_95 = np.mean(total_costs[total_costs >= var_95])
        cvar_99 = np.mean(total_costs[total_costs >= var_99])
        
        strategy_results[strategy_name] = {
            'generation': gen,
            'costs': total_costs,
            'mean_cost': mean_cost,
            'std_cost': std_cost,
            'var_95': var_95,
            'var_99': var_99,
            'cvar_95': cvar_95,
            'cvar_99': cvar_99,
            'color': strategy_params['color']
        }
    
    # 1. 効率フロンティア（リスク-リターン）
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # リスク-リターン散布図
    for strategy_name, results in strategy_results.items():
        ax1.scatter(results['std_cost'], results['mean_cost'], 
                   s=200, c=results['color'], alpha=0.7, label=strategy_name)
        ax1.annotate(strategy_name, 
                    (results['std_cost'], results['mean_cost']),
                    xytext=(10, 10), textcoords='offset points')
    
    ax1.set_xlabel('Risk (Standard Deviation) [$]')
    ax1.set_ylabel('Expected Return (Mean Cost) [$]')
    ax1.set_title('Risk-Return Analysis')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. コスト分布の比較
    for strategy_name, results in strategy_results.items():
        ax2.hist(results['costs'], bins=50, alpha=0.6, 
                label=strategy_name, color=results['color'], density=True)
    
    ax2.set_xlabel('Total Cost [$]')
    ax2.set_ylabel('Probability Density')
    ax2.set_title('Cost Distribution Comparison')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. VaR/CVaR比較
    strategies_list = list(strategy_results.keys())
    var_95_values = [strategy_results[s]['var_95'] for s in strategies_list]
    var_99_values = [strategy_results[s]['var_99'] for s in strategies_list]
    cvar_95_values = [strategy_results[s]['cvar_95'] for s in strategies_list]
    cvar_99_values = [strategy_results[s]['cvar_99'] for s in strategies_list]
    
    x_pos = np.arange(len(strategies_list))
    width = 0.2
    
    ax3.bar(x_pos - 1.5*width, var_95_values, width, label='VaR 95%', alpha=0.7)
    ax3.bar(x_pos - 0.5*width, var_99_values, width, label='VaR 99%', alpha=0.7)
    ax3.bar(x_pos + 0.5*width, cvar_95_values, width, label='CVaR 95%', alpha=0.7)
    ax3.bar(x_pos + 1.5*width, cvar_99_values, width, label='CVaR 99%', alpha=0.7)
    
    ax3.set_xlabel('Strategy')
    ax3.set_ylabel('Risk Measure [$]')
    ax3.set_title('VaR and CVaR Comparison')
    ax3.set_xticks(x_pos)
    ax3.set_xticklabels(strategies_list)
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Violin plot（分布形状の詳細比較）
    cost_data_for_violin = []
    strategy_labels_for_violin = []
    
    for strategy_name, results in strategy_results.items():
        cost_data_for_violin.extend(results['costs'])
        strategy_labels_for_violin.extend([strategy_name] * len(results['costs']))
    
    violin_df = pd.DataFrame({
        'Strategy': strategy_labels_for_violin,
        'Cost': cost_data_for_violin
    })
    
    sns.violinplot(data=violin_df, x='Strategy', y='Cost', ax=ax4)
    ax4.set_title('Cost Distribution Shapes (Violin Plot)')
    ax4.set_ylabel('Total Cost [$]')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # リスク指標の表
    risk_summary = []
    for strategy_name, results in strategy_results.items():
        risk_summary.append({
            'Strategy': strategy_name,
            'Generation': results['generation'],
            'Mean_Cost': results['mean_cost'],
            'Std_Cost': results['std_cost'],
            'VaR_95': results['var_95'],
            'CVaR_95': results['cvar_95'],
            'VaR_99': results['var_99'],
            'CVaR_99': results['cvar_99']
        })
    
    risk_df = pd.DataFrame(risk_summary)
    print("\n=== リスク分析結果要約 ===")
    display(risk_df.round(2))
    
    return strategy_results

# リスク分析実行
risk_analysis_results = create_advanced_risk_analysis()

## 4. 時系列データの可視化とトレンド分析

In [ ]:
def create_time_series_analysis():
    """
    時系列データの可視化と分析
    """
    # 時系列データの生成（1年間の日次データ）
    np.random.seed(456)
    dates = pd.date_range('2023-01-01', '2023-12-31', freq='D')
    n_days = len(dates)
    
    # 季節性を含む風力データ生成
    day_of_year = np.array([d.timetuple().tm_yday for d in dates])
    seasonal_pattern = 10 * np.cos(2 * np.pi * day_of_year / 365) + 50
    wind_output = seasonal_pattern + np.random.normal(0, 8, n_days)
    wind_output = np.maximum(0, wind_output)  # 負の値を除去
    
    # 需要データ（季節変動 + 週次パターン）
    base_demand = 80 + 15 * np.cos(2 * np.pi * day_of_year / 365)
    weekly_pattern = 5 * np.sin(2 * np.pi * np.arange(n_days) / 7)
    demand = base_demand + weekly_pattern + np.random.normal(0, 5, n_days)
    
    # 最適発電量の時系列計算
    optimal_generation = []
    daily_costs = []
    
    for i in range(n_days):
        # 簡化された日次最適化
        expected_wind = wind_output[i]
        daily_demand = demand[i]
        
        # ヒューリスティック最適解
        opt_gen = max(0, daily_demand - expected_wind)
        optimal_generation.append(opt_gen)
        
        # 日次コスト
        shortage = max(0, daily_demand - opt_gen - expected_wind)
        daily_cost = 60 * opt_gen + 200 * shortage
        daily_costs.append(daily_cost)
    
    # データフレーム作成
    time_series_df = pd.DataFrame({
        'Date': dates,
        'Wind_Output': wind_output,
        'Demand': demand,
        'Optimal_Generation': optimal_generation,
        'Daily_Cost': daily_costs,
        'Month': [d.month for d in dates],
        'DayOfWeek': [d.dayofweek for d in dates],
        'Quarter': [d.quarter for d in dates]
    })
    
    # 1. 基本的な時系列プロット
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 12))
    
    # 風力・需要・発電量の時系列
    ax1.plot(time_series_df['Date'], time_series_df['Wind_Output'], 
             alpha=0.7, label='Wind Output', color='blue', linewidth=1)
    ax1.plot(time_series_df['Date'], time_series_df['Demand'], 
             alpha=0.7, label='Demand', color='red', linewidth=1)
    ax1.plot(time_series_df['Date'], time_series_df['Optimal_Generation'], 
             alpha=0.7, label='Conventional Generation', color='green', linewidth=1)
    
    ax1.set_ylabel('Power [MW]')
    ax1.set_title('Daily Power Time Series')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # コストの時系列
    ax2.plot(time_series_df['Date'], time_series_df['Daily_Cost'], 
             color='purple', alpha=0.7, linewidth=1)
    
    # 移動平均線を追加
    rolling_cost = time_series_df['Daily_Cost'].rolling(window=30).mean()
    ax2.plot(time_series_df['Date'], rolling_cost, 
             color='black', linewidth=2, label='30-day Moving Average')
    
    ax2.set_ylabel('Daily Cost [$]')
    ax2.set_title('Daily Cost Evolution')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 季節性分析（月別ボックスプロット）
    monthly_data = []
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    for month in range(1, 13):
        month_data = time_series_df[time_series_df['Month'] == month]['Wind_Output']
        monthly_data.append(month_data)
    
    bp = ax3.boxplot(monthly_data, labels=month_names, patch_artist=True)
    
    # ボックスプロットに色を付ける
    colors = plt.cm.viridis(np.linspace(0, 1, 12))
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    ax3.set_ylabel('Wind Output [MW]')
    ax3.set_title('Seasonal Wind Output Patterns')
    ax3.grid(True, alpha=0.3)
    
    # 週次パターン分析
    day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    weekly_costs = []
    
    for day in range(7):
        day_data = time_series_df[time_series_df['DayOfWeek'] == day]['Daily_Cost']
        weekly_costs.append(day_data)
    
    bp2 = ax4.boxplot(weekly_costs, labels=day_names, patch_artist=True)
    
    colors2 = plt.cm.Set3(np.linspace(0, 1, 7))
    for patch, color in zip(bp2['boxes'], colors2):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    ax4.set_ylabel('Daily Cost [$]')
    ax4.set_title('Weekly Cost Patterns')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # インタラクティブな時系列グラフ（Plotly）
    fig_interactive = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Power Variables', 'Daily Cost'),
        vertical_spacing=0.1
    )
    
    # 電力変数
    fig_interactive.add_trace(
        go.Scatter(x=time_series_df['Date'], y=time_series_df['Wind_Output'],
                  mode='lines', name='Wind Output', line=dict(color='blue')),
        row=1, col=1
    )
    
    fig_interactive.add_trace(
        go.Scatter(x=time_series_df['Date'], y=time_series_df['Demand'],
                  mode='lines', name='Demand', line=dict(color='red')),
        row=1, col=1
    )
    
    fig_interactive.add_trace(
        go.Scatter(x=time_series_df['Date'], y=time_series_df['Optimal_Generation'],
                  mode='lines', name='Generation', line=dict(color='green')),
        row=1, col=1
    )
    
    # コスト
    fig_interactive.add_trace(
        go.Scatter(x=time_series_df['Date'], y=time_series_df['Daily_Cost'],
                  mode='lines', name='Daily Cost', line=dict(color='purple')),
        row=2, col=1
    )
    
    fig_interactive.add_trace(
        go.Scatter(x=time_series_df['Date'], y=rolling_cost,
                  mode='lines', name='30-day MA', line=dict(color='black', width=3)),
        row=2, col=1
    )
    
    fig_interactive.update_xaxes(title_text="Date", row=2, col=1)
    fig_interactive.update_yaxes(title_text="Power [MW]", row=1, col=1)
    fig_interactive.update_yaxes(title_text="Cost [$]", row=2, col=1)
    
    fig_interactive.update_layout(
        title='Interactive Time Series Analysis',
        width=1000, height=800,
        hovermode='x unified'
    )
    
    fig_interactive.show()
    
    # 統計サマリー
    print("\n=== 時系列統計サマリー ===")
    summary_stats = time_series_df[['Wind_Output', 'Demand', 'Optimal_Generation', 'Daily_Cost']].describe()
    display(summary_stats.round(2))
    
    # 相関分析
    correlation_matrix = time_series_df[['Wind_Output', 'Demand', 'Optimal_Generation', 'Daily_Cost']].corr()
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
                square=True, fmt='.3f', cbar_kws={'label': 'Correlation Coefficient'})
    plt.title('Variable Correlation Matrix')
    plt.tight_layout()
    plt.show()
    
    return time_series_df

# 時系列分析実行
time_series_data = create_time_series_analysis()

## 5. 意思決定支援ダッシュボード

In [ ]:
def create_decision_support_dashboard():
    """
    総合的な意思決定支援ダッシュボード
    """
    # ダッシュボード用のサマリーデータ
    np.random.seed(789)
    
    # KPIデータ
    kpi_data = {
        'Current Strategy': 'Moderate',
        'Expected Annual Cost': '$2,435,000',
        'Cost Savings vs Conservative': '$285,000',
        'VaR (95%)': '$3,120,000',
        'CVaR (95%)': '$3,456,000',
        'Risk Premium': '$1,021,000',
        'Uptime': '98.7%',
        'Shortage Events': '12 days/year'
    }
    
    # 複数戦略の比較
    strategy_comparison = pd.DataFrame({
        'Strategy': ['Conservative', 'Moderate', 'Aggressive'],
        'Expected_Cost': [2720, 2435, 2180],
        'VaR_95': [3200, 3120, 3580],
        'CVaR_95': [3400, 3456, 4120],
        'Sharpe_Ratio': [0.65, 0.71, 0.58],
        'Shortage_Days': [3, 12, 28]
    })
    
    # レーダーチャート用データ
    radar_categories = ['Cost Efficiency', 'Risk Control', 'Reliability', 
                       'Flexibility', 'Sustainability']
    
    radar_values = {
        'Conservative': [60, 95, 98, 70, 85],
        'Moderate': [85, 80, 94, 85, 90],
        'Aggressive': [95, 55, 85, 95, 75]
    }
    
    # ダッシュボード作成
    print("\n" + "="*60)
    print("         電力系統運用 意思決定支援ダッシュボード")
    print("="*60)
    
    # KPI表示
    print("\n📊 主要業績指標 (KPI)")
    print("-" * 40)
    for key, value in kpi_data.items():
        print(f"{key:.<30} {value:>15}")
    
    # 戦略比較表
    print("\n\n📈 戦略比較分析")
    print("-" * 40)
    display(strategy_comparison)
    
    # 可視化部分
    fig = make_subplots(
        rows=2, cols=3,
        subplot_titles=('Strategy Performance', 'Risk Measures', 'Cost Breakdown',
                       'Risk-Return Efficiency', 'Reliability Metrics', 'Strategy Radar'),
        specs=[[{'type': 'bar'}, {'type': 'bar'}, {'type': 'pie'}],
               [{'type': 'scatter'}, {'type': 'bar'}, {'type': 'scatterpolar'}]],
        horizontal_spacing=0.1,
        vertical_spacing=0.15
    )
    
    # 1. 戦略パフォーマンス
    fig.add_trace(
        go.Bar(x=strategy_comparison['Strategy'], 
               y=strategy_comparison['Expected_Cost'],
               name='Expected Cost',
               marker_color=['#FF6B6B', '#4ECDC4', '#45B7D1']),
        row=1, col=1
    )
    
    # 2. リスク指標
    fig.add_trace(
        go.Bar(x=strategy_comparison['Strategy'], 
               y=strategy_comparison['VaR_95'],
               name='VaR 95%', 
               marker_color='lightblue'),
        row=1, col=2
    )
    
    fig.add_trace(
        go.Bar(x=strategy_comparison['Strategy'], 
               y=strategy_comparison['CVaR_95'],
               name='CVaR 95%', 
               marker_color='darkblue'),
        row=1, col=2
    )
    
    # 3. コスト分解（パイチャート）
    cost_breakdown = ['Conventional Generation', 'Emergency Power', 'Fixed Costs']
    cost_values = [1800, 435, 200]
    fig.add_trace(
        go.Pie(labels=cost_breakdown, values=cost_values, name="Cost Breakdown"),
        row=1, col=3
    )
    
    # 4. リスク-リターン効率
    fig.add_trace(
        go.Scatter(x=strategy_comparison['Expected_Cost'], 
                   y=strategy_comparison['VaR_95'],
                   mode='markers+text',
                   text=strategy_comparison['Strategy'],
                   textposition='top center',
                   marker=dict(size=12, color=['red', 'green', 'blue']),
                   name='Risk-Return'),
        row=2, col=1
    )
    
    # 5. 信頼性指標
    fig.add_trace(
        go.Bar(x=strategy_comparison['Strategy'], 
               y=strategy_comparison['Shortage_Days'],
               name='Shortage Days',
               marker_color='orange'),
        row=2, col=2
    )
    
    # 6. レーダーチャート
    for strategy, values in radar_values.items():
        fig.add_trace(
            go.Scatterpolar(
                r=values + [values[0]],  # 閉じた図形のため最初の値を追加
                theta=radar_categories + [radar_categories[0]],
                fill='toself',
                name=strategy
            ),
            row=2, col=3
        )
    
    # レイアウト更新
    fig.update_layout(
        title_text="⚡ 電力系統運用 総合ダッシュボード",
        title_x=0.5,
        showlegend=True,
        width=1400,
        height=900
    )
    
    # 軸ラベルの設定
    fig.update_xaxes(title_text="Strategy", row=1, col=1)
    fig.update_yaxes(title_text="Cost [$1000]", row=1, col=1)
    
    fig.update_xaxes(title_text="Strategy", row=1, col=2)
    fig.update_yaxes(title_text="Risk Measure [$1000]", row=1, col=2)
    
    fig.update_xaxes(title_text="Expected Cost [$1000]", row=2, col=1)
    fig.update_yaxes(title_text="VaR 95% [$1000]", row=2, col=1)
    
    fig.update_xaxes(title_text="Strategy", row=2, col=2)
    fig.update_yaxes(title_text="Shortage Days/Year", row=2, col=2)
    
    fig.show()
    
    # 推奨事項
    print("\n\n💡 戦略推奨事項")
    print("-" * 40)
    
    recommendations = [
        "🎯 現在の中庸戦略は、コスト効率とリスク管理のバランスが良好",
        "⚠️  VaRとCVaRの差（1,021K$）は適切な範囲内",
        "📈 年間12日の不足イベントは業界標準内",
        "🔍 風力予測精度向上により、さらなるコスト削減が期待",
        "📊 四半期ごとの戦略見直しを推奨",
        "🔄 需要パターン変化に応じた動的調整を検討"
    ]
    
    for rec in recommendations:
        print(rec)
    
    # アクションアイテム
    print("\n\n📋 アクションアイテム")
    print("-" * 40)
    
    actions = [
        "1. 風力予測システムの精度評価と改善計画策定",
        "2. 緊急電源調達コストの契約条件見直し", 
        "3. 需要予測における機械学習手法の導入検討",
        "4. リアルタイム最適化システムの性能向上",
        "5. 近隣系統との電力融通契約の拡張検討"
    ]
    
    for action in actions:
        print(action)
    
    return strategy_comparison

# 意思決定支援ダッシュボード作成
dashboard_data = create_decision_support_dashboard()

print("\n" + "="*60)
print("           ダッシュボード分析完了")
print("="*60)

## 6. まとめ：可視化のベストプラクティス

In [ ]:
def visualization_best_practices():
    """
    確率計画法の可視化ベストプラクティス
    """
    print("\n" + "="*60)
    print("      確率計画法可視化のベストプラクティス")
    print("="*60)
    
    practices = {
        "1. データ理解": [
            "• 分布の特性（平均、分散、歪度、尖度）を必ず可視化",
            "• ヒストグラム、密度プロット、Q-Qプロットの併用",
            "• 外れ値の検出と影響評価",
            "• 時系列データの季節性・トレンド分析"
        ],
        "2. リスク可視化": [
            "• VaR/CVaRを直感的に表示（色分け、閾値表示）",
            "• リスク-リターンの散布図とパレートフロンティア",
            "• 複数信頼水準での比較",
            "• Violin plotで分布の形状詳細を表示"
        ],
        "3. 最適化結果": [
            "• パラメータ感度のヒートマップ",
            "• 3D可視化による多次元最適化ランドスケープ",
            "• 収束過程のアニメーション",
            "• 制約の活性状況の可視化"
        ],
        "4. 意思決定支援": [
            "• KPIダッシュボードで重要指標を一目で把握",
            "• レーダーチャートで多角的評価",
            "• インタラクティブな要素で詳細分析を可能に",
            "• シナリオ比較のための並列座標プロット"
        ],
        "5. 技術的考慮": [
            "• 色覚多様性への配慮（カラーパレット選択）",
            "• レスポンシブデザインで様々な画面サイズに対応",
            "• データの更新頻度に応じたリアルタイム表示",
            "• エクスポート機能（PDF、PNG、データ）の実装"
        ]
    }
    
    for category, items in practices.items():
        print(f"\n{category}")
        print("-" * len(category))
        for item in items:
            print(item)
    
    print("\n\n📚 推奨ライブラリ・ツール")
    print("-" * 30)
    
    tools = {
        "静的可視化": "Matplotlib, Seaborn, Plotly (静的)",
        "インタラクティブ可視化": "Plotly, Bokeh, Altair",
        "ダッシュボード": "Dash, Streamlit, Panel",
        "Web展開": "Flask + Plotly, FastAPI + Charts",
        "レポート生成": "Jupyter + nbconvert, R Markdown"
    }
    
    for tool_type, tools_list in tools.items():
        print(f"{tool_type:.<25} {tools_list}")
    
    print("\n\n🎯 学習成果")
    print("-" * 30)
    
    achievements = [
        "✓ 確率分布の多角的可視化手法の習得",
        "✓ リスク分析のための高度なグラフ作成",
        "✓ 最適化結果の効果的な表現方法",
        "✓ 時系列データの包括的分析手法",
        "✓ 意思決定支援のためのダッシュボード設計",
        "✓ MatplotlibとPlotlyの使い分け",
        "✓ インタラクティブ要素の効果的活用",
        "✓ 実務に即した可視化設計思想"
    ]
    
    for achievement in achievements:
        print(achievement)
    
    print("\n\n🚀 次のステップ")
    print("-" * 30)
    
    next_steps = [
        "1. 実際のデータでの可視化実践",
        "2. Webアプリケーションとしての展開",
        "3. 自動レポート生成システムの構築", 
        "4. リアルタイム監視ダッシュボードの開発",
        "5. 機械学習結果の可視化手法の拡張"
    ]
    
    for step in next_steps:
        print(step)

# ベストプラクティス表示
visualization_best_practices()

print("\n\n🎉 可視化・グラフ分析ノートブック完了！")
print("確率計画法の結果を効果的に可視化し、")
print("意思決定支援のための包括的な分析手法を習得しました。")